In [4]:
import pandas as pd
import numpy as np

csv_path1 = "students_complete.csv"
csv_path2 = 'schools_complete.csv'
students_df = pd.read_csv(csv_path1)
schools_df = pd.read_csv(csv_path2)

students_df['user_key'] = students_df['school'].map(lambda x: x.lower())
schools_df['user_key'] = schools_df['name'].map(lambda x: x.lower())

joined_df = pd.merge(schools_df, students_df, how='left', on='user_key')
joined_df.head()

,School ID,name_x,type,size,budget,user_key,Student ID,name_y,gender,grade,school,reading_score,math_score
0,0,Huang High School,District,2917,1910635,huang high school,0,Paul Bradley,M,9th,Huang High School,66,79
1,0,Huang High School,District,2917,1910635,huang high school,1,Victor Smith,M,12th,Huang High School,94,61
2,0,Huang High School,District,2917,1910635,huang high school,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,0,Huang High School,District,2917,1910635,huang high school,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,0,Huang High School,District,2917,1910635,huang high school,4,Bonnie Ray,F,9th,Huang High School,97,84


In [5]:
#District Summary
total_schools = joined_df.name_x.nunique()
total_students = joined_df.name_y.nunique()
total_budget = joined_df.budget.sum()
avg_math_score = round(joined_df.math_score.mean(), 2)
avg_reading_score = round(joined_df.reading_score.mean(), 2)

def score_passed(input_score):
    if input_score >= 70:
        return "pass"
    else:
        return "fail"

joined_df['math_pass'] = joined_df['math_score'].map(score_passed)
joined_df['reading_pass'] = joined_df['reading_score'].map(score_passed)



math_passed = joined_df.loc[joined_df["math_score"]> 70,:]
percent_passed_math = round((math_passed["Student ID"].count() / total_students),2) * 100
reading_passed = students_df.loc[students_df["reading_score"]> 70,:]
percent_passed_reading = round((reading_passed["Student ID"].count() / total_students),2) * 100
overall_passing_rate = (percent_passed_math + percent_passed_reading)/2

district_summary = pd.DataFrame({"Total Schools": [total_schools], 
                              "Total Students": [total_students], 
                              "Total Budget": [total_budget], 
                              "Average Math Score": [avg_math_score], 
                              "Average Reading Score": [avg_reading_score], 
                              "Percentage Passing Math": [percent_passed_math], 
                              "Percentage Passing Reading": [percent_passed_reading], 
                              "Overall Passing Rate": [overall_passing_rate]})
district_summary.head()


,Average Math Score,Average Reading Score,Overall Passing Rate,Percentage Passing Math,Percentage Passing Reading,Total Budget,Total Schools,Total Students
0,78.99,81.88,93.0,87.0,99.0,82932329558,15,32715


In [7]:
#School Summary
schools_df = schools_df.rename(columns={"name":"school_name"})
students_df = students_df.rename(columns={"school":"school_name"})

merge_table = pd.merge(schools_df, students_df, on="school_name")
merge_table.head()

sch_avg_scores = pd.DataFrame(students_df.groupby("school_name").mean())
sch_avg_scores = sch_avg_scores.reset_index()
sch_avg_scores = sch_avg_scores.drop('Student ID', axis=1)

school_summary_df = schools_df
school_summary_df = school_summary_df.merge(sch_avg_scores,on="school_name")
school_summary_df["Per Student Budget"] = round(school_summary_df["budget"] / school_summary_df["size"])

math_passed_table = students_df.loc[students_df["math_score"]> 70,:]
math_passed_totals = math_passed_table.groupby("school_name")['Student ID'].count()   
math_passed_totals = math_passed_totals.reset_index()
math_passed_totals = math_passed_totals.rename(columns={'Student ID':'math_passed'})

read_passed_table = students_df.loc[students_df["reading_score"]> 70,:]
read_passed_table.head()
read_passed_totals = read_passed_table.groupby("school_name")['Student ID'].count()
read_passed_totals = read_passed_totals.reset_index()
read_passed_totals = read_passed_totals.rename(columns={'Student ID':'read_passed'})

school_summary_df = school_summary_df.merge(math_passed_totals,on="school_name")
school_summary_df = school_summary_df.merge(read_passed_totals,on="school_name")
school_summary_df['math_passed'] = (school_summary_df['math_passed'] / school_summary_df["size"]) * 100 
school_summary_df['read_passed'] = (school_summary_df['read_passed'] / school_summary_df["size"]) * 100
school_summary_df['% Overall Passing'] = (school_summary_df['math_passed'] + school_summary_df['read_passed'])/2

school_summary_df = school_summary_df.rename(columns= {"math_passed": "% Passing Math",
                                            "read_passed": "% Passing Reading",
                                                       "school_name": "School Name",
                                                       "type": "School Type",
                                                       "size": "Total Students",
                                                       "budget": "Total School Budget",
                                                       "reading_score": "Average Reading Score",
                                                       "math_score": "Average Math Score"})
                                     
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].map('${:,.2f}'.format)
school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].map('${:,.2f}'.format)

school_summary_df = school_summary_df[["School Name",
                              "School Type",
                              "Total Students",
                              "Total School Budget",
                                "Per Student Budget",
                              "Average Math Score",
                              "Average Reading Score",
                              "% Passing Math",
                              "% Passing Reading",
                              "% Overall Passing"]]
school_summary_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
2,Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,89.892107,92.617831,91.254969
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867


In [8]:
#Top 5 Schools
sorted_df = school_summary_df.sort_values(["% Overall Passing"], ascending=[False])
top_five = sorted_df.head()
top_five

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867
10,Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,90.277778,93.444444,91.861111
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


In [9]:
#Bottom 5 Schools
sorted_df = school_summary_df.sort_values(["% Overall Passing"], ascending=[True])
sorted_df.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
11,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098


In [10]:
#Math Scores by Grade
grouped = joined_df.groupby(['school','grade'])
math_df = pd.DataFrame(grouped['math_score'].mean())
math_df = math_df.reset_index()
pivot_math = math_df.pivot(index='school',columns='grade',values='math_score')
pivot_math = pivot_math.reset_index()

pivot_math.columns.name = None
pivot_math = pivot_math[['school','9th','10th','11th','12th']]
pivot_math.set_index('school', inplace=True)
pivot_math

,9th,10th,11th,12th
school,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [11]:
#Reading Scores by Grade
read_df = pd.DataFrame(grouped['reading_score'].mean())
read_df = read_df.reset_index()
pivot_read = read_df.pivot(index='school',columns='grade',values='reading_score')
pivot_read = pivot_read.reset_index()

pivot_read.columns.name = None
pivot_read = pivot_read[['school','9th','10th','11th','12th']]
pivot_read.set_index('school', inplace=True)
pivot_read

,9th,10th,11th,12th
school,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [12]:
#Scores by School Spending
schools_df = school_summary_df
schools_df['Spending Range'] = schools_df['Per Student Budget'].replace('\$','',regex=True).astype('float')

bins = [0,585,610,635,660]
group_names = ['Less than $585','\$575-$610','\$610-$635','More than $660']

group_indices = ["1","2","3","4"]

schools_df["Spending Range Bins"] = pd.cut(schools_df["Spending Range"], bins, labels=group_indices)

school_groups = schools_df.groupby("Spending Range Bins")
school_means = school_groups.mean()
school_means.drop('Total Students', axis=1,inplace=True)
school_means.drop('Spending Range', axis=1,inplace=True)
school_means.reset_index(inplace=True)

school_means['Spending Range Bins'] = school_means['Spending Range Bins'].replace(['1','2','3','4'], group_names)
school_means.set_index('Spending Range Bins', inplace=True)
school_means

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Range Bins,,,,,
Less than $585,83.455399,83.933814,90.350436,93.325838,91.838137
\$575-$610,83.599686,83.885211,90.788049,92.410786,91.599418
\$610-$635,80.199966,82.425360,77.172061,86.346507,81.759284
More than $660,77.866721,81.368774,67.957362,80.268067,74.112715


In [14]:
#Scores by School Size
bins = [0,1500,2500,5000]
group_names = ['Small (<1500)','Medium (1500-2500)','Large (>2500)']
group_indices = ["1","2","3"]
schools_df["Size Bins"] = pd.cut(schools_df["Total Students"], bins, labels=group_indices)

school_groups = schools_df.groupby("Size Bins")
school_means = school_groups.mean()
school_means.drop('Total Students', axis=1,inplace=True)
school_means.reset_index(inplace=True)
school_means

,Size Bins,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Range
0,1,83.664898,83.892148,90.676736,92.778720,91.727728,605.000000
1,2,83.359224,83.898984,90.175120,93.217267,91.696193,596.200000
2,3,76.956733,80.966636,64.302528,78.324559,71.313543,643.571429


In [15]:
#Scores by School Type
school_groups = schools_df.groupby("School Type")
school_means = school_groups.mean()
school_means

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Range
School Type,,,,,,,
Charter,1524.250000,83.473852,83.896421,90.363226,93.052812,91.708019,599.500000
District,3853.714286,76.956733,80.966636,64.302528,78.324559,71.313543,643.571429


#3 Observable trends based on the data:
### 1. On average, the charter schools performed much better than the district schools
### 2. On average, the smaller schools performed better than the larger schools.
### 3. On average, the schools that spent the most per student, had the lowest passing scores.